# Projeto Automação Web (Selenium)

#### Etapas do projeto:
1. Importar bibliotecas e planilha
2. Percorrer cada linha da planilha
3. Pegar preço de cada loja
4. Registrar o menor preço
5. Registrar local de menor preço
6. Salvar planilha e enviar e-mail caso exista algum produto com pelo menos 20% de desconto

Obs: Os links utilizados neste projeto podem sofrer alterações

### Importando bibliotecas

In [6]:
import pandas as pd
from selenium import webdriver
import yagmail

produtos = pd.read_excel(r'C:\Users\jp_si\Downloads\Projeto Automação Web\Produtos.xlsx')
produtos = produtos.fillna('-')
display(produtos)



,Link Produto,Amazon,Lojas Americanas,Magazine Luiza,Preço Original,Preço Atual,Local
0,iPhone 11 Apple 64GB Preto,https://www.amazon.com.br/IPHONE-11-PRETO-TELA...,https://www.americanas.com.br/produto/16113159...,https://www.magazineluiza.com.br/iphone-11-app...,5297,-,-
1,Smart TV LED 50'' LG Ultra HD 4K Thinq AI,https://www.amazon.com.br/Smart-LG-Intelig%C3%...,https://www.americanas.com.br/produto/49516150...,https://www.magazineluiza.com.br/smart-tv-uhd-...,2900,-,-


In [ ]:
def transformar_texto(texto):
    return float(texto.replace('R$', '').replace('.', '').replace(',', '.'))

In [11]:
driver = webdriver.Chrome()
driver.set_window_position(-10000,0)

enviar_email = False
desconto_min = 0.2

for i, linha in produtos.iterrows():
    #pegar preço amazon
    driver.get(linha['Amazon'])
    try:
        preco_amazon = driver.find_element_by_id('priceblock_ourprice').text
        preco_amazon = transformar_texto(preco_amazon)
    except:
        try:
            preco_amazon = driver.find_element_by_css_selector('#olp-upd-new span.a-size-base').text
            preco_amazon = transformar_texto(preco_amazon)
        except:
            print('Produto {} não disponível na Amazon'.format(linha['Link Produto']))
            preco_amazon = linha['Preço Original'] * 3
    

    #pegar o preço da lame
    driver.get(linha['Lojas Americanas'])
    try:
        preco_lame = driver.find_element_by_class_name('src__BestPrice-lo2vta-5').text
        preco_lame = transformar_texto(preco_lame)
    except:
        print('Produto {} não disponível na Lojas Americanas'.format(linha['Link Produto']))
        preco_lame = linha['Preço Original'] * 3
        
    #pegar o preço da mglu
    driver.get(linha['Magazine Luiza'])
    try:
        preco_mglu = driver.find_element_by_class_name('price-template__text').text
        preco_mglu = transformar_texto(preco_mglu)
    except:
        print('Produto {} não disponível na Magazine Luiza'.format(linha['Link Produto']))
        preco_mglu = linha['Preço Original'] * 3

    preco_original = linha['Preço Original']
    
    lista_precos = [(preco_amazon, 'Amazon'), (preco_lame, 'Lojas Americanas'), (preco_mglu, 'Magazine Luiza'), (preco_original, 'Original')]
    
    lista_precos.sort()
    
    produtos.loc[i, 'Preço Atual'] = lista_precos[0][0]
    produtos.loc[i, 'Local'] = lista_precos[0][1]
    
    if lista_precos[0][0] <= preco_original*(1-desconto_min):
        enviar_email = True
    

### Enviar E-mail

In [ ]:
usuario = yagmail.SMTP(user='jpsilveirabastos@gmail.com', password='*********')

#salvar o arquivo
produtos.to_excel('C:\Users\jp_si\Downloads\Projeto Automação Web\Produtos.xlsx')

if enviar_email:
    email = 'chinatradebr@gmail.com'
    assunto = f'Produto(s) Encontrado(s) com mais de {desconto_min:.0%} de Desconto'
    
     #filtrar a tabela de produtos
    tabela_filtrada = produtos.loc[produtos['Preço Atual'] <= produtos['Preço Original']*(1-desconto_min), :]

    texto = f'<p>Esses sãos os produtos com mais de {desconto_min:.0%} de desconto</p>{tabela_filtrada.to_html()}'
    
    usuario.send(to=email, subject=assunto, contents=texto)
    
driver.quit()